<a href="https://colab.research.google.com/github/SuHamza/private-ai/blob/master/Section%203%20-%20Securing%20Federated%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project 1: Federated Learning with a Trusted Aggregator

In [0]:
# Project 1: Federated Learning with a Trusted Aggregator

In [2]:
!pip install syft==0.1.21a1

     |████████████████████████████████| 225kB 5.0MB/s 
     |████████████████████████████████| 389kB 39.6MB/s 
     |████████████████████████████████| 256kB 37.9MB/s 
     |████████████████████████████████| 204kB 41.5MB/s 
     |████████████████████████████████| 2.1MB 28.7MB/s 
     |████████████████████████████████| 460kB 41.7MB/s 
     |████████████████████████████████| 81kB 26.8MB/s 
     |████████████████████████████████| 276kB 42.9MB/s 
     |████████████████████████████████| 51kB 20.2MB/s 
     |████████████████████████████████| 122kB 37.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
   

In [3]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0725 16:36:05.150778 139985144219520 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0725 16:36:05.171393 139985144219520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# Create Workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")  # Trusted 3rd party

In [5]:
# In real-world workers
# Need to inform workers that other workers exist
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0725 16:36:09.694201 139985144219520 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0725 16:36:09.696321 139985144219520 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0725 16:36:09.697680 139985144219520 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0725 16:36:09.699615 139985144219520 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0725 16:36:09.702160 139985144219520 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0725 16:36:09.703783 139985144219520 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [0]:
# Get pointers to training data on each worker
# by sending first part of data to bob & second part to alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [0]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
# Initialize Toy model on Central Server
model = nn.Linear(2,1)

In [20]:
for round_iter in range(10):

  # Send a copy of the model to both workers
  bobs_model = model.copy().send(bob)
  alices_model = model.copy().send(alice)

  # Create two optimizers
  bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
  alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

  for i in range(10):
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target) **2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    bobs_loss

    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target) **2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss

  # Move models from both worker to secure_worker
  alices_model.move(secure_worker)
  bobs_model.move(secure_worker)

  # Average both models
  avg_weight = (alices_model.weight.data + bobs_model.weight.data) / 2
  # Pull resulting average to Local machine
  avg_weight = avg_weight.get()
  #model.weight.data.set_(avg_weight) ## Causes RuntimeError: set_storage is not allowed on Tensor created from .data or .detach()
  model.weight.data = avg_weight

  # Average Bias
  avg_bias = (alices_model.bias.data + bobs_model.bias.data) / 2
  avg_bias = avg_bias.get()

  model.bias.data = avg_bias
  model.bias.data
  
  print("Bob's Loss: " + str(bobs_loss) + " Alice's Loss: " + str(alices_loss))

Bob's Loss: tensor(1.3876e-06) Alice's Loss: tensor(0.0074)
Bob's Loss: tensor(0.0008) Alice's Loss: tensor(0.0026)
Bob's Loss: tensor(0.0013) Alice's Loss: tensor(0.0011)
Bob's Loss: tensor(0.0014) Alice's Loss: tensor(0.0005)
Bob's Loss: tensor(0.0012) Alice's Loss: tensor(0.0003)
Bob's Loss: tensor(0.0010) Alice's Loss: tensor(0.0002)
Bob's Loss: tensor(0.0007) Alice's Loss: tensor(0.0001)
Bob's Loss: tensor(0.0006) Alice's Loss: tensor(0.0001)
Bob's Loss: tensor(0.0004) Alice's Loss: tensor(7.5651e-05)
Bob's Loss: tensor(0.0003) Alice's Loss: tensor(5.6307e-05)


In [14]:
secure_worker._objects

{31175208013: Parameter containing:
 tensor([0.4719], requires_grad=True), 36984618295: Parameter containing:
 tensor([-0.0471], requires_grad=True), 80592667372: Parameter containing:
 tensor([[0.4281, 0.1781]], requires_grad=True), 87730342543: Parameter containing:
 tensor([[-0.2191,  0.0761]], requires_grad=True)}

In [21]:
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

---



In [23]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [24]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [25]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [26]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [28]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project 2: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
# Project 2: Build Methods for Encrypt, Decrypt, and Add

In [0]:
import random

In [0]:
Q = 23740629843760239486723

In [0]:
def encrypt(x, n_shares=3):
  shares = list()
  for i in range(n_shares - 1):
    shares.append(random.randint(0,Q))
    
  final_share = Q - (sum(shares) % Q) + x
  shares.append(final_share)
  
  return tuple(shares)

In [34]:
encrypt(5)

(12584502952176301944235, 23732898580778641960036, 11163858154565535069180)

In [0]:
def decrypt(shares):
  return sum(shares) % Q

In [36]:
decrypt(encrypt(5))

5

In [0]:
def add(a, b):
  c = list()
  
  assert(len(a) == len(b))
  
  for i in range(len(a)):
    c.append((a[i] + b[i]) % Q)
    
  return tuple(c)

In [39]:
decrypt(add(encrypt(5), encrypt(10)))

15

In [44]:
x = encrypt(5)
x

(18955367674115924303135, 11728993829428518555845, 16796898183976036114471)

In [45]:
y = encrypt(10)
y

(19152196866370197300946, 15586212964178675338640, 12742849856971606333870)

In [46]:
z = add(x, y)
z

(14366934696725882117358, 3574576949846954407762, 5799118197187402961618)

In [47]:
decrypt(z)

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [0]:
encode(3.5)

35000

In [0]:
decode(35000)

3.5

In [0]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice, secure_worker)

In [0]:
bob._objects

{35498656553: tensor([  10235770278698899, 1401398179551373756, 2277280072169145491,
          636965538565031298,  913795591610271305])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [0]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:23637986557 -> bob:30254176063]
	-> (Wrapper)>[PointerTensor | me:18229131498 -> alice:75856222543]
	-> (Wrapper)>[PointerTensor | me:34301722959 -> secure_worker:75419815101]
	*crypto provider: me*

In [0]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [0]:
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [0]:
x.child.child

tensor([100, 200, 300])

In [0]:
y = x + x

In [0]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [0]:
y = x + x

In [0]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation